## For reference (var1)

In [ ]:
# liquid sectional area
def A_l (var1):
    return A*var1
# gas sectional area
def A_g (var1):
    return A*(1 - var1)
# stratification angle
def gamma (var1):
    return pi*(var1) + pow ((3.*pi/2.), (1./3.))*(1 - 2*(var1) + pow (abs (var1), (1./3.)) - pow (abs(1 - (var1)), (1/3))) - (1/200)*(1 - (var1))*(var1)*(1 - 2*(var1))*(1+4*((1 - (var1))**2 + (var1)**2))
# def gamma (var1):
#     return pi*(var1 + DOLFIN_EPS) + pow ((3.*pi/2.), (1./3.))*(1 - 2*(var1 + DOLFIN_EPS) + pow (abs (var1 + DOLFIN_EPS), (1./3.)) - pow (abs(1 - (var1 + DOLFIN_EPS)), (1/3))) - (1/200)*(1 - (var1 + DOLFIN_EPS))*(var1 + DOLFIN_EPS)*(1 - 2*(var1 + DOLFIN_EPS))*(1+4*((1 - (var1 + DOLFIN_EPS))**2 + (var1 + DOLFIN_EPS)**2))

# liquid wetted perimeter
def P_lw (var1):
    return D*gamma (var1)
# gas wetted perimeter
def P_gw (var1):
    return D*(pi - gamma (var1))
# interface wetted perimeter
def P_gl (var1):
    return D*np.sin (gamma (var1))
# liquid hydraulic diameter 
def Dh_l (var1):
    return 4.*A_l (var1)/P_lw (var1)
# gas hydraulic diameter 
def Dh_g (var1):
    return 4.*A_g (var1)/(P_gw (var1) + P_gl (var1))
#Gas density
def rho_g (var4):
    return p_factor*var4
# def rho_g (var4):
#     return p_factor*var4 + DOLFIN_EPS

# liquid phase velocity
def var2 (var1):
    return j_l/(var1)
# gas phase velocity
def var3 (var1):
    return j_g/(1 - var1)
# liquid Reynolds number
def Re_l (var1, var2):
    return rho_l*var2*Dh_l (var1)/mu_l
# gas Reynolds number
def Re_g (var1, var3, var4):
    return rho_g (var4)*var3*Dh_g (var1)/mu_g
# interface hydraulic diameter 
def Dh_gl (var1):
    return 4.*A_g (var1)/(P_gw (var1) + P_gl (var1))
# interface Reynolds number
def Re_gl (var1, var2, var3, var4):
    return rho_g (var4)*abs(var3 - var2)*Dh_gl (var1)/mu_g
# Friction factor liquid-wall
def f_lw (var1, var2):
    return 0.046*abs(Re_l (var1, var2))**(-0.2)
# Friction factor gas-wall
def f_gw (var1, var3, var4):
    return 0.046*abs(Re_g (var1, var3, var4))**(-0.2)
# interfacial friction
def f_gl (var1, var2, var3, var4):
    return max (f_gw (var1, var3, var4), 0.014)
# Shear stress liquid-wall
def tau_lw (var1, var2):
    return 1/2*f_lw (var1, var2)*rho_l*var2*abs (var2)
# gas shear stress
def tau_gw (var1, var3, var4):
    return 1/2*f_gw (var1, var3, var4)*rho_g (var4)*var3*abs (var3)
# interface shear stress l
def tau_gl (var1, var2, var3, var4):
    return 1/2*f_gl(var1, var2, var3, var4)*rho_g (var4)*(var3 - var2)*abs (var3 - var2)

# Equilibrium
def equilibrium1 (var1): 
    return (rho_g (var4_0) - rho_l)*g*np.sin (beta) - tau_lw (var1, var2 (var1))*P_lw (var1)/A_l (var1) + tau_gw (var1, var3 (var1), var4_0)*P_gw (var1)/A_g (var1) + tau_gl (var1, var2 (var1), var3 (var1), var4_0)*P_gl (var1)*(1/A_l (var1) + 1/A_g (var1))

## For reference (j_l)

In [ ]:
# liquid phase velocity
def var2b (j_l):
    return j_l/(var1)
# Equilibrium
def equilibrium2 (j_l):  
    return (rho_g (var4_0) - rho_l)*g*np.sin (beta) - tau_lw (var1, var2b (j_l))*P_lw (var1)/A_l (var1) + tau_gw (var1, var3 (var1), var4_0)*P_gw (var1)/A_g (var1) + tau_gl (var1, var2b (j_l), var3 (var1), var4_0)*P_gl (var1)*(1/A_l (var1) + 1/A_g (var1))

## 1. Reference state
find var1

FUNCTION:
ref, Re_ref = ref_state (j_l, j_g, var4_0, beta, rho_l, p_factor, mu_l, mu_g, D)

In [ ]:
def ref_state (j_l, j_g, var4_0, beta, rho_l, p_factor, mu_l, mu_g, D):
    # Reference state
    var1_ref = brenth (equilibrium1, lima, limb)
    var2_ref = j_l/var1_ref
    var3_ref = j_g/(1 - var1_ref)
    var4_ref = var4_0

    ref = np.array ([var1_ref, var2_ref, var3_ref, var4_ref])
    
    # Reynolds number of initial conditions
    Rel_ref = Re_l (var1_ref, var2_ref)
    Reg_ref = Re_g (var1_ref, var3_ref, var4_ref)
    Regl_ref = Re_gl (var1_ref, var2_ref, var3_ref, var4_ref)

    # Reynolds reference conditions
    Re_ref = np.array ([Rel_ref, Reg_ref, Regl_ref])

    return ref, Re_ref

## 2. Reference state
Find j_l

In [ ]:
def ref_state_jl (var1, j_g, var4_0, beta, rho_l, p_factor, mu_l, mu_g, D):
    # Define j_l
    j_l = fsolve (equilibrium2, x0) #Python function    

    var1_ref = var1
    var2_ref = j_l[0]/var1_ref
    var3_ref = j_g/(1 - var1_ref)
    var4_ref = var4_0

    return np.array ([var1_ref, var2_ref, var3_ref, var4_ref])

## For matrices

In [ ]:
# critic diameter
def Dc (var1):
    return g*np.cos (beta)*pi*D/(4*np.sin (gamma (var1)))

## For FEniCS

In [ ]:
# def P_gl_fenics (var1):
#     return D*sin (gamma (var1))

# def Dh_g_fenics (var1):
#     return 4.*A_g (var1)/(P_gw (var1) + P_gl_fenics (var1))

# def Dh_gl_fenics (var1):
#     return 4.*A_g (var1)/(P_gw (var1) + P_gl_fenics (var1))

# def Dc_fenics (var1):
#     return g*cos (beta)*pi*D/(4*sin (gamma (var1)))

# def Re_g_fenics (var1, var3, var4):
#     return rho_g (var4)*var3*Dh_g_fenics (var1)/mu_g

# def Re_gl_fenics (var1, var2, var3, var4):
#     return rho_g (var4)*abs(var3 - var2)*Dh_gl_fenics (var1)/mu_g

# def f_lw_fenics (var1, var2):
#     Af_lw = pow (2.457*ln (pow (pow (abs (7/Re_l (var1, var2)), 0.9) + 0.27*roughness/Dh_l (var1), -1)), 16)
#     Bf_lw = pow (abs (37530/Re_l (var1, var2)), 16)
#     return 2*pow (pow (abs (8/Re_l (var1, var2)), 12) + pow (abs (Af_lw + Bf_lw), - 1.5), 1/12)

# def f_gw_fenics (var1, var3, var4):
#     Af_gw = pow (2.457*ln (pow (pow (abs (7/Re_g_fenics (var1, var3, var4)), 0.9) + 0.27*roughness/Dh_g_fenics (var1), - 1)), 16)
#     Bf_gw = pow (abs (37530/Re_g_fenics (var1, var3, var4)), 16)
#     return 2*pow (pow (abs (8/Re_g_fenics (var1, var3, var4)), 12) + pow (abs (Af_gw + Bf_gw), - 1.5), 1/12)

# def f_gl_fenics (var1, var2, var3, var4):
#     return max (f_gw_fenics (var1, var3, var4), 0.014)

# def tau_gw_fenics (var1, var3, var4):
#     return 1/2*f_gw_fenics (var1, var3, var4)*rho_g (var4)*var3*abs (var3)

# def tau_gl_fenics (var1, var2, var3, var4):
#     return 1/2*f_gl_fenics (var1, var2, var3, var4)*rho_g (var4)*(var3 - var2)*abs (var3 - var2)
 